# 🚀 Entorno completo n8n + Google Drive en Google Colab
Autor: **David Moya**  
Fecha: **Octubre 2025**  

Este notebook permite ejecutar **n8n** directamente desde **Google Colab**, con:
- Persistencia en Google Drive
- Autenticación básica (usuario/contraseña)
- Backups automáticos diarios
- Restauración y exportación a ZIP
- Informe visual (HTML + TXT) de los flujos

📂 Estructura generada en tu Drive:
```
MyDrive/
└── n8n_data/
    ├── .n8n/
    ├── backups/
    ├── exports/
    └── reports/
```

In [ ]:
# ============================================
# 1️⃣ INSTALACIÓN Y CONFIGURACIÓN BÁSICA
# ============================================
from google.colab import drive
import os

print("🔗 Montando Google Drive...")
drive.mount('/content/drive')

# Crear carpeta persistente
!mkdir -p /content/drive/MyDrive/n8n_data/.n8n

# Instalar Node.js 18 y n8n
print("⚙️ Instalando Node.js y n8n...")
!curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
!apt install -y nodejs
!npm install -g n8n

# Configuración de autenticación básica
os.environ['N8N_BASIC_AUTH_ACTIVE'] = 'true'
os.environ['N8N_BASIC_AUTH_USER'] = 'admin'
os.environ['N8N_BASIC_AUTH_PASSWORD'] = '1234'
os.environ['N8N_USER_FOLDER'] = '/content/drive/MyDrive/n8n_data/.n8n'
os.environ['N8N_DEFAULT_LOCALE'] = 'es'

print("✅ Configuración lista. Ejecutando n8n...")
print("🌐 Una vez iniciado, se mostrará un enlace tipo: https://xxxxx.loca.lt")

!n8n start --tunnel

In [ ]:
# ============================================
# 2️⃣ BACKUP AUTOMÁTICO DE FLUJOS A GOOGLE DRIVE
# ============================================
import datetime
drive.mount('/content/drive', force_remount=True)
backup_dir = "/content/drive/MyDrive/n8n_data/backups"
os.makedirs(backup_dir, exist_ok=True)
fecha = datetime.datetime.now().strftime("%Y-%m-%d")
backup_file = f"{backup_dir}/backup_{fecha}.json"
print("📤 Exportando flujos de n8n...")
!n8n export:workflow --all --output="{backup_file}"
print(f"✅ Backup completado: {backup_file}")

In [ ]:
# ============================================
# 3️⃣ RESTAURAR BACKUP DESDE GOOGLE DRIVE
# ============================================
drive.mount('/content/drive', force_remount=True)
backup_dir = "/content/drive/MyDrive/n8n_data/backups"
print("📂 Backups disponibles:")
!ls -lh "$backup_dir" | grep ".json" || echo "No hay backups disponibles."
backup_to_restore = "backup_2025-10-30.json"
backup_path = f"{backup_dir}/{backup_to_restore}"
if os.path.exists(backup_path):
    print(f"🔄 Restaurando backup: {backup_to_restore}")
    !n8n import:workflow --input="{backup_path}" --separate
    print("✅ Restauración completada.")
else:
    print("❌ Error: archivo no encontrado.")

In [ ]:
# ============================================
# 4️⃣ EXPORTAR Y COMPRIMIR FLUJOS A ZIP
# ============================================
import datetime
drive.mount('/content/drive', force_remount=True)
base_dir = "/content/drive/MyDrive/n8n_data"
export_dir = f"{base_dir}/exports"
os.makedirs(export_dir, exist_ok=True)
fecha = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
json_file = f"{export_dir}/workflows_{fecha}.json"
zip_file = f"{export_dir}/n8n_export_{fecha}.zip"
print("📤 Exportando flujos...")
!n8n export:workflow --all --output="{json_file}"
print("🗜️ Comprimiendo...")
!zip -j "{zip_file}" "{json_file}"
print(f"✅ ZIP generado en Drive: {zip_file}")

In [ ]:
# ============================================
# 5️⃣ INFORME VISUAL (HTML + TXT)
# ============================================
import datetime, json
from IPython.display import HTML, display
drive.mount('/content/drive', force_remount=True)
base_dir = "/content/drive/MyDrive/n8n_data/reports"
os.makedirs(base_dir, exist_ok=True)
fecha = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M")
json_temp = f"/content/workflows_temp_{fecha}.json"
html_report = f"{base_dir}/informe_workflows_{fecha}.html"
txt_report = f"{base_dir}/informe_workflows_{fecha}.txt"
print("📤 Exportando flujos para generar informe...")
!n8n export:workflow --all --output="{json_temp}"
with open(json_temp, 'r') as f:
    data = json.load(f)
html_content = f"<h2>📊 Informe de Workflows n8n ({fecha})</h2>"
html_content += "<table border='1' cellspacing='0' cellpadding='6'><tr><th>ID</th><th>Nombre</th><th>Activo</th><th>Fecha Creación</th><th>Última actualización</th></tr>"
txt_content = f"📊 INFORME DE WORKFLOWS n8n ({fecha})\n\n"
txt_content += f"{'ID':<10} | {'Nombre':<35} | {'Activo':<8} | {'Creado':<20} | {'Actualizado':<20}\n"
txt_content += "-"*110 + "\n"
for wf in data:
    wf_id = wf.get('id', 'N/A')
    name = wf.get('name', 'Sin nombre')
    active = '✅ Sí' if wf.get('active', False) else '❌ No'
    created = wf.get('createdAt', 'N/D').replace('T', ' ').replace('Z', '')
    updated = wf.get('updatedAt', 'N/D').replace('T', ' ').replace('Z', '')
    html_content += f"<tr><td>{wf_id}</td><td>{name}</td><td>{active}</td><td>{created}</td><td>{updated}</td></tr>"
    txt_content += f"{wf_id:<10} | {name:<35} | {active:<8} | {created:<20} | {updated:<20}\n"
html_content += "</table>"
with open(html_report, 'w') as f:
    f.write(html_content)
with open(txt_report, 'w') as f:
    f.write(txt_content)
print(f"✅ Informe generado:\n- HTML: {html_report}\n- TXT: {txt_report}")
display(HTML(html_content))